<a href="https://colab.research.google.com/github/EddieJabraouti/Transformer/blob/main/gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-07-13 20:00:59--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  3.97MB/s    in 0.3s    

2025-07-13 20:01:00 (3.97 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open("input.txt", "r", encoding="utf-8") as f:
  text = f.read()

In [3]:
print(len(text))

1115394


In [4]:
#looking at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
#all the unique characters in the data set
chars = sorted(list(set(text))) #creates the text into a set and then makes a sorted list out of each character alphabetically.
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
#create a mapping for the characters to ints

stoi = {ch: i for i,ch in enumerate(chars)} #individualy maps each character to an integer for all characters in chars
itos = {i:ch for i,ch in enumerate(chars)} #individual maps each integer to its designated characters for all ints in chars
encode = lambda s: [stoi[c] for c in s] #encoder takes a string and ouputs it as a list of integers
decoder = lambda l: ''.join([itos[i] for i in l]) #decoder takes a list of integers and outputs a string
print(encode("Hello"))
print(decoder(encode("Hello")))

[20, 43, 50, 50, 53]
Hello


In [7]:
#lets encode the entire data set and store it in a torch.tensor matrix

import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
#seperating data into training and validation sets before we fit it to a minature LLM model

n = int(0.9*len(data)) #first 90% of the data set will be for training, the rest for validation
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target is: ", target)

when input is tensor([18]) the target is:  tensor(47)
when input is tensor([18, 47]) the target is:  tensor(56)
when input is tensor([18, 47, 56]) the target is:  tensor(57)
when input is tensor([18, 47, 56, 57]) the target is:  tensor(58)
when input is tensor([18, 47, 56, 57, 58]) the target is:  tensor(1)
when input is tensor([18, 47, 56, 57, 58,  1]) the target is:  tensor(15)
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is:  tensor(47)
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is:  tensor(58)


In [11]:
torch.manual_seed(1337) #torch.manual_seed() is a method for randomly generating values, each time being the same values

batch_size = 4 #number of independent sequences that will be procssed in parallel
block_size = 8 #what is the mac context length for predicitons

def get_batch(split):
  #generate a small batch of data for inputs x and targets y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix]) #torch.stack takes each consecutive chunk of the data set and concatenates it along a new dimension
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print("Targets:")
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when input is {context.tolist()}, output is {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24], output is 43
when input is [24, 43], output is 58
when input is [24, 43, 58], output is 5
when input is [24, 43, 58, 5], output is 57
when input is [24, 43, 58, 5, 57], output is 1
when input is [24, 43, 58, 5, 57, 1], output is 46
when input is [24, 43, 58, 5, 57, 1, 46], output is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43], output is 39
when input is [44], output is 53
when input is [44, 53], output is 56
when input is [44, 53, 56], output is 1
when input is [44, 53, 56, 1], output is 58
when input is [44, 53, 56, 1, 58], output is 46
when input is [44, 53, 56, 1, 58, 

In [12]:
print("Our inputs to the transformer: \n", xb)

Our inputs to the transformer: 
 tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [13]:
#implenting the Bigram Language model
#Bigram model is a statistical neural network that calculates the probabilty of a word given the preceeding word in the sequence

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) #creates an embedding table full of raw scores correlating to the probability of a word. Each row is a word that has a corresponding vector to it

  def forward(self, idx, targets=None):
      logits = self.token_embedding_table(idx) #logits will correlate the the vector at the idx

      if targets is None:
        loss = None
      else:
        B,T,C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        #measure the quality of predictions
        loss = F.cross_entropy(logits, targets)   #given we have the target how well have we predicted it based on logits


      return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      #get the predictions
      logits, loss = self(idx)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      #sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1)
      #append sampled index to the running sequence
      idx = torch.cat((idx, idx_next), dim=1)
    return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decoder(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [14]:
#Create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [15]:
batch_size = 32
for steps in range(10000):
  #Sample a batch of data
  xb, yb = get_batch('train')

  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.5727508068084717


In [16]:
print(decoder(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y helti


###Self attention - utilizing Matrix Multiplication

In [17]:
torch.manual_seed(1337)
B,T,C = 4,8,2 #Batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [18]:
#We want x[b,t] = mean_{i<=t} x[b, i]
#bow = bag of words

xbow = torch.zeros((B,T,C))
for b in range (B):
  for t in range(T):
    xprev = x[b, : t+1] #(t, c)
    xbow[b, t] = torch.mean(xprev, 0) #torch.mean(input tensor, dimension) - dimension corresponds to either the Batch(0), Time(1), Channels(2)
                                      #in this case we are averging out the 4 batches

In [19]:
#wei => weights

wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x #(B,T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2, rtol=1e-3, atol=1e-4)

True

In [20]:
torch.manual_seed(42)
a = torch.ones(3, 3)
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3, 2)).float()
c = a @ b
print("a = ", a)
print("b = ", b)
print("c = ", c)



a =  tensor([[0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333]])
b =  tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c =  tensor([[4.6667, 5.3333],
        [4.6667, 5.3333],
        [4.6667, 5.3333]])


In [21]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3, rtol=1e-3, atol=1e-4)

True

In [22]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) #(B, T, 16)
q = query(x) #(B, T, 16)

wei = q @ k.transpose(-2, -1) #(B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T, T)) #torch.tril is a PyTorch function that returns the lower triangular part of a 2-D tensor (matrix) or a batch of matrices.
#wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x
print(out.shape)

torch.Size([4, 8, 16])


In [23]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

 Notes:
 <ul>
  <li>
    Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at eachother and aggregating information with a weighted sum from all nodes that point to them, with data dependant weights
  </li>
  <li>
    There is no notion of space. Attention simply acts over a set of vectotrs. This is why we need to positionally encode tokens.
  </li>
  <li>
    "self-attention" just mean that the keys and values are produced from the source as queries. In "cross attention", the queries still gets produced form x, but the keys and values come from some other external sours(e.g, and encoder module)
  <li>
    In an Encoder attention block just delete the single line that does masking with <code>Trill</code>, allowing all tokens to communicate. This Block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
  </li>
  <li>
    Scaling the attentiona divides wei by 1/sqrt(head_size). This makes it so that the variance is minimal at when initializing.
  </li>
 </ul>

In [27]:
#Batch layer normalization: imporve the volatility of values so that training the nn will be more efficient
class Batch:
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    xmean = x.mean(1, keepdim=True) #the batch mean
    xvar = x.var(1, keepdim=True) #the batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) #normialize unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]
torch.manual_seed(1337)
module = Batch(100)
x = torch.randn(32, 100) #batch size of 32 of 100 dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])